In [9]:
import numpy as np

np.show_config()

Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/arm64-builds/include
    lib directory: /opt/arm64-builds/lib
    name: openblas64
    openblas configuration: USE_64BITINT=1 DYNAMIC_ARCH=1 DYNAMIC_OLDER= NO_CBLAS=
      NO_LAPACK= NO_LAPACKE= NO_AFFINITY=1 USE_OPENMP= SANDYBRIDGE MAX_THREADS=3
    pc file directory: /usr/local/lib/pkgconfig
    version: 0.3.23.dev
  lapack:
    detection method: internal
    found: true
    include directory: unknown
    lib directory: unknown
    name: dep4350285776
    openblas configuration: unknown
    pc file directory: unknown
    version: 1.26.4
Compilers:
  c:
    args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: cc
    linker: ld64
    linker args: -fno-strict-aliasing, -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    name: clang
    version: 14.0.0
  c++:
    args: -DBLAS_SYMBOL_SUFFIX=64_, -DHAVE_BLAS_ILP64
    commands: c++
    linker: ld64
    linker

# side model

In [10]:
import pandas as pd

df_features = pd.read_parquet("data/features_1h.parquet")
label_side = np.load("data/label_side_1h.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(44867, 11841)
(44867, 7)


4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_ac_23                        0
1h_ac_22                        0
1h_mod_stochastic               0
1h_trendflex                    0
1h_ac_31                        0
Length: 11841, dtype: int64

In [11]:
df_features = df_features[364:]
label_side = label_side[364:]

print(df_features.shape)
print(label_side.shape)

label_side

(44503, 11841)
(44503, 7)


array([[ 1.58666418e+12,  6.81173000e+03,  6.91519000e+03, ...,
         6.77566000e+03,  7.38005150e+04,  1.00000000e+00],
       [ 1.58670006e+12,  6.91519000e+03,  7.08500000e+03, ...,
         6.90500000e+03,  7.22904840e+04,  1.00000000e+00],
       [ 1.58670972e+12,  7.08500000e+03,  7.11225000e+03, ...,
         7.04632000e+03,  7.13427380e+04,  1.00000000e+00],
       ...,
       [ 1.73826948e+12,  1.05165000e+05,  1.04909800e+05, ...,
         1.04768000e+05,  4.98372300e+03, -1.00000000e+00],
       [ 1.73827092e+12,  1.04909800e+05,  1.05189300e+05, ...,
         1.04737000e+05,  4.83362100e+03, -1.00000000e+00],
       [ 1.73827926e+12,  1.05189200e+05,  1.04416800e+05, ...,
         1.04356100e+05,  4.86977900e+03, -1.00000000e+00]])

In [12]:
from custom_indicators.config import SIDE_ALL

side_features = df_features[SIDE_ALL]
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(44503, 2368)
(array([0, 1]), array([20772, 23731]))
(array([0, 1]), array([3872, 5029]))


In [5]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-18 23:13:17,121] A new study created in memory with name: no-name-a6f6d109-755b-4734-a090-042f17cc9485
[I 2025-03-18 23:13:50,703] Trial 0 finished with value: 0.8244636685735134 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 97, 'max_depth': 130, 'min_gain_to_split': 0.014752222678836285, 'min_data_in_leaf': 470, 'lambda_l1': 40.207358986972935, 'lambda_l2': 74.3810285439051, 'num_boost_round': 1188}. Best is trial 0 with value: 0.8244636685735134.


precision: 0.7971046282079404, recall: 0.7226088685623384, f1: 0.7580308719232374


[I 2025-03-18 23:14:07,183] Trial 1 finished with value: 0.8193383848883089 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 185, 'max_depth': 48, 'min_gain_to_split': 0.2246313230716316, 'min_data_in_leaf': 116, 'lambda_l1': 37.691387441560614, 'lambda_l2': 84.54294546444926, 'num_boost_round': 900}. Best is trial 0 with value: 0.8244636685735134.


precision: 0.7940133037694014, recall: 0.7120699940345994, f1: 0.7508124541356537


[I 2025-03-18 23:14:29,118] Trial 2 finished with value: 0.8293781398467401 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 217, 'max_depth': 799, 'min_gain_to_split': 0.5830611051043544, 'min_data_in_leaf': 81, 'lambda_l1': 91.64724581155298, 'lambda_l2': 69.64528645412508, 'num_boost_round': 1049}. Best is trial 2 with value: 0.8293781398467401.


...

precision: 0.7917301414581066, recall: 0.723404255319149, f1: 0.756026600166251


[I 2025-03-18 23:35:38,441] Trial 98 finished with value: 0.8361482738957023 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 118, 'max_depth': 184, 'min_gain_to_split': 0.5870519524126964, 'min_data_in_leaf': 445, 'lambda_l1': 3.884738174590418, 'lambda_l2': 84.88012849631176, 'num_boost_round': 208}. Best is trial 98 with value: 0.8361482738957023.


precision: 0.7993124194241513, recall: 0.7397096838337641, f1: 0.7683569141794898


[I 2025-03-18 23:35:46,004] Trial 99 finished with value: 0.8340288003135534 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 116, 'max_depth': 193, 'min_gain_to_split': 0.4619928895034619, 'min_data_in_leaf': 386, 'lambda_l1': 2.9074977174466454, 'lambda_l2': 75.93385067634634, 'num_boost_round': 195}. Best is trial 98 with value: 0.8361482738957023.


precision: 0.7983906046107003, recall: 0.7299661960628355, f1: 0.7626467227589072


In [7]:
study.best_params

{'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 118,
 'max_depth': 184,
 'min_gain_to_split': 0.5870519524126964,
 'min_data_in_leaf': 445,
 'lambda_l1': 3.884738174590418,
 'lambda_l2': 84.88012849631176,
 'num_boost_round': 208}

In [17]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    # **study.best_params,
    "extra_trees": True,
    "boosting": "dart",
    "num_leaves": 118,
    "max_depth": 184,
    "min_gain_to_split": 0.5870519524126964,
    "min_data_in_leaf": 445,
    "lambda_l1": 3.884738174590418,
    "lambda_l2": 84.88012849631176,
    "num_boost_round": 208,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [18]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [20]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8361482738957023


In [21]:
model.save_model("custom_indicators/models/model_side.txt")
prod_model.save_model("custom_indicators/models/model_side_prod.txt")

# meta model

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from custom_indicators.config import SIDE_ALL

df_features = pd.read_parquet("data/features_1h.parquet")
meta_label = np.load("data/label_meta_1h.npy")

model_side = lgb.Booster(model_file="custom_indicators/models/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features.isna().sum(axis=0).sort_values(ascending=False)

4h_dft_dom_cycle_ddt_lag39    364
4h_dft_dom_cycle_ddt_lag38    360
4h_dft_dom_cycle_dt_lag39     360
4h_dft_dom_cycle_ddt_lag37    356
4h_dft_dom_cycle_lag39        356
                             ... 
1h_comb_spectrum_pwr_14         0
1h_comb_spectrum_pwr_13         0
1h_comb_spectrum_pwr_12         0
1h_comb_spectrum_pwr_11         0
side_model_res                  0
Length: 11842, dtype: int64

In [2]:
df_features = df_features[364:]
meta_label = meta_label[364:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

(44503, 11842)
(44503, 7)


In [3]:
from custom_indicators.config import META_ALL

meta_features = df_features[META_ALL]
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(44503, 2368)
(array([0, 1]), array([ 6573, 29029]))
(array([0, 1]), array([2243, 6658]))


In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.25, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-19 12:14:35,940] A new study created in memory with name: no-name-bbcf5450-3649-4cc1-bb74-8b4ebcc67f03
[I 2025-03-19 12:15:16,887] Trial 0 finished with value: 0.7030905548089867 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 248, 'max_depth': 987, 'min_gain_to_split': 0.12374740165582301, 'min_data_in_leaf': 97, 'lambda_l1': 38.811848464500336, 'lambda_l2': 1.3336949974469667, 'num_boost_round': 679}. Best is trial 0 with value: 0.7030905548089867.


precision: 0.7917144495412844, recall: 0.8295283869029738, f1: 0.8101804312747543, auc: 0.6915475628794473, fbeta: 0.7030905548089867


[I 2025-03-19 12:15:56,586] Trial 1 finished with value: 0.7025611418865645 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 245, 'max_depth': 978, 'min_gain_to_split': 0.01684185618749966, 'min_data_in_leaf': 55, 'lambda_l1': 51.8651209848161, 'lambda_l2': 12.38973802913599, 'num_boost_round': 815}. Best is trial 0 with value: 0.7030905548089867.


precision: 0.7903064861012117, recall: 0.8326824872334034, f1: 0.8109412711182623, auc: 0.6899516629755106, fbeta: 0.7025611418865645


[I 2025-03-19 12:17:54,101] Trial 2 finished with value: 0.6874351786499537 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 143, 'max_depth': 780, 'min_gain_to_split': 0.45267652028523303, 'min_data_in_leaf': 293, 'lambda_l1': 87.27412776045, 'lambda_l2': 45.52291533881712, 'num_boost_round': 1820}. Best is trial 0 with value: 0.7030905548089867.


...

precision: 0.8043942630454685, recall: 0.7918293781916491, f1: 0.7980623675446564, auc: 0.7011813529679533, fbeta: 0.7019721677717469


[I 2025-03-19 13:16:11,177] Trial 98 finished with value: 0.6986405227885131 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 339, 'max_depth': 980, 'min_gain_to_split': 0.06556243264344014, 'min_data_in_leaf': 93, 'lambda_l1': 10.294223116335113, 'lambda_l2': 51.50173088547197, 'num_boost_round': 1471}. Best is trial 70 with value: 0.7055773589800834.


precision: 0.7737226277372263, recall: 0.8915590267347552, f1: 0.8284717376133984, auc: 0.6875782029790757, fbeta: 0.6986405227885131


[I 2025-03-19 13:16:46,044] Trial 99 finished with value: 0.7045010710244515 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 313, 'max_depth': 949, 'min_gain_to_split': 0.41119181322166004, 'min_data_in_leaf': 167, 'lambda_l1': 16.397361393280278, 'lambda_l2': 30.390177106703863, 'num_boost_round': 1697}. Best is trial 70 with value: 0.7055773589800834.


precision: 0.7860008299903167, recall: 0.8534094322619405, f1: 0.8183192914236336, auc: 0.6920599543561781, fbeta: 0.7045010710244515


In [5]:
study.best_params

{'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 344,
 'max_depth': 841,
 'min_gain_to_split': 0.1314742604013666,
 'min_data_in_leaf': 58,
 'lambda_l1': 46.192695758687805,
 'lambda_l2': 41.064614815829444,
 'num_boost_round': 1540}

In [6]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [7]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [8]:
model.save_model("custom_indicators/models/model_meta.txt")
prod_model.save_model("custom_indicators/models/model_meta_prod.txt")